In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [2]:
training_data_uri = 's3://pwalanki-blunder-predictor-data/output-medium'


In [3]:
!pygmentize 'model.py'

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import argparse
import time
import os
import json


def model(training_data, validation_data, dense_layers, num_nodes, epochs, learning_rate, dropout, callback=None):
	feature_columns = [
		tf.feature_column.numeric_column('position', shape=(1, 8, 8, 12), dtype=tf.dtypes.int64),
		tf.feature_column.numeric_column('turn', shape=(1,), dtype=tf.dtypes.int64),
		tf.feature_column.numeric_column('elo', shape=(1,), dtype=tf.dtypes.float32)
	]

	model = tf.keras.models.Sequential()
	model.add(tf.keras.layers.DenseFeatures(feature_columns))

	for i in range(dense_layers):
		model.add(tf.keras.layers.Dense(num_nodes, activation=tf.nn.relu))

	model.add(tf.keras.layers.Dropout(dropout))

	model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

	model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
				  loss='binary_crossentropy',
				  metrics=['accuracy', tf.metrics.Rec

In [4]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='model.py',
                       role=role,
                       train_instance_count=1,
                       train_instance_type='ml.m4.xlarge',
                       framework_version='2.1.0',
                       py_version='py3',
                       hyperparameters={'aws': True})

In [5]:
estimator.fit(training_data_uri)

2020-04-07 21:50:02 Starting - Starting the training job...
2020-04-07 21:50:04 Starting - Launching requested ML instances.........
2020-04-07 21:51:35 Starting - Preparing the instances for training......
2020-04-07 21:52:40 Downloading - Downloading input data......
2020-04-07 21:53:59 Training - Training image download completed. Training in progress...2020-04-07 21:54:06,738 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-04-07 21:54:06,745 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-07 21:54:07,279 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-07 21:54:07,298 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-07 21:54:07,318 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-07 21:54:07,330 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_param

2020-04-07 21:54:31.207193: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015 1/35 [..............................] - ETA: 7s - loss: 0.6679 - accuracy: 0.5720 - recall: 0.4511 - precision: 0.5963#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/35 [>.............................] - ETA: 5s - loss: 0.6680 - accuracy: 0.5815 - recall: 0.5100 - precision: 0.5928#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

2020-04-07 21:54:39.386614: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015 1/35 [..............................] - ETA: 6s - loss: 0.6514 - accuracy: 0.6270 - recall: 0.6806 - precision: 0.6155#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/35 [>.............................] - ETA: 5s - loss: 0.6497 - accuracy: 0.6255 - recall: 0.7146 - precision: 0.6100#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

2020-04-07 21:54:55.619968: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015 1/35 [..............................] - ETA: 6s - loss: 0.6282 - accuracy: 0.6330 - recall: 0.7475 - precision: 0.6104#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/35 [>.............................] - ETA: 5s - loss: 0.6304 - accuracy: 0.6265 - recall: 0.7679 - precision: 0.5955#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

2020-04-07 21:55:03.767993: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015 1/35 [..............................] - ETA: 7s - loss: 0.6227 - accuracy: 0.6450 - recall: 0.7706 - precision: 0.6138#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/35 [>.............................] - ETA: 6s - loss: 0.6238 - accuracy: 0.6395 - recall: 0.7944 - precision: 0.6038#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

2020-04-07 21:55:11.923430: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015 1/35 [..............................] - ETA: 7s - loss: 0.6226 - accuracy: 0.6410 - recall: 0.7639 - precision: 0.6160#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/35 [>.............................] - ETA: 5s - loss: 0.6218 - accuracy: 0.6360 - recall: 0.7843 - precision: 0.6069#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

2020-04-07 21:55:20.656855: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015 1/35 [..............................] - ETA: 8s - loss: 0.6174 - accuracy: 0.6460 - recall: 0.7660 - precision: 0.6296#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/35 [>.............................] - ETA: 6s - loss: 0.6164 - accuracy: 0.6405 - recall: 0.7932 - precision: 0.6063#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

2020-04-07 21:55:36.972598: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015 1/35 [..............................] - ETA: 7s - loss: 0.6181 - accuracy: 0.6360 - recall: 0.7714 - precision: 0.6091#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/35 [>.............................] - ETA: 5s - loss: 0.6114 - accuracy: 0.6450 - recall: 0.7946 - precision: 0.6126#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

2020-04-07 21:55:41.521069: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
#011 [[{{node IteratorGetNext}}]]
#015      1/Unknown - 0s 191ms/step - loss: 0.6098 - accuracy: 0.6540 - recall: 0.8063 - precision: 0.6252#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015      2/Unknown - 0s 164ms/step - loss: 0.6046 - accuracy: 0.6515 - recall: 0.8006 - precision: 0.6177#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01


2020-04-07 21:55:53 Uploading - Uploading generated training model
2020-04-07 21:55:53 Completed - Training job completed
Training seconds: 193
Billable seconds: 193
